In [553]:
from elasticsearch import Elasticsearch
from datetime import datetime, timedelta

import json
cluster = ['ip-172-31-0-107', 'ip-172-31-0-100', 'ip-172-31-0-105', 'ip-172-31-0-106']
es = Elasticsearch(cluster, port=9200)

In [365]:
def dispatch(ctime, location, driver, name, p1=None, p2=None):
        #d = retrieveDriver(driver, es)
        
        p = scanPassenger(location, es)
        
        dDoc = {"ctime": ctime, "location": location, "status": "idle"}
        if p:
            doc = {"status": "pickup", "driver": driver, "ctime": ctime}
            updatePassenger(p['id'], doc, es)

            if p1: 
                dDoc['p2'] = p['id']
            else:
                dDoc['p1'] = p['id']
                
            dDoc['status'] = "pickup"
            dDoc['destination'] = p['location']
            dDoc['altdest1'] = p['id']
            

        updateDriver(driver, dDoc, es)
        
        bulk = (1, '{{doc: {}}}'.format(json.dumps(dDoc)))
        return (bulk)

In [ ]:
      'driver': {'type': 'string', "include_in_all": "false"},
        'destination': {'type': 'geo_point', 'lat_lon': 'true', "include_in_all": "false"},
        'destinationid':{'type': 'string', "include_in_all": "false"},                
        'altdest1': {'type': 'geo_point', 'lat_lon': 'true', "include_in_all": "false"},
        'altdestid1':{'type': 'string', "include_in_all": "false"},                    
        'altdest2': {'type': 'geo_point', 'lat_lon': 'true', "include_in_all": "false"},
        'altdestid2':{'type': 'string', "include_in_all": "false"}, 
        'origin': {'type': 'geo_point', 'lat_lon': 'true', "include_in_all": "false"},
        'path': {'type': 'string', "include_in_all": "false"}


In [540]:
window = 'now-2h'
location =[40.7589, -73.985]

def retrievePassenger(pID, es):
    p = es.get(index='passenger', doc_type='rolling', id=pID, \
                   ignore=[404, 400])
    return p['_source'] if p['found'] else False


p = retrievePassenger(44, es)
if p: 
    shoulds = []
    for i in [p['destinationid'],p['altdest1id'],p['altdest1id']]:
        shoulds.append({'match': {'destinationid': i}})
        shoulds.append({'match': {'altdest1id': i}}) 
        shoulds.append({'match': {'altdest1id': i}}) 

    destinations = [p['destinationid'], p['altdest1id'], p['altdest2id']]
    geo_query = {"size": 1, 
         "query" : {
          "bool" : {
          "must" : { "term" : { "status" : "arrived" }},
      "must_not" : { "term" : { "id" : p['id'] }},

         "filter": {
        "geo_distance": {
            "distance": '3km',
       "distance_type": "plane", 
            "location": location }},

        "should" : shoulds,
      "minimum_should_match" : 1,
                     "boost" : 1.0
            }},
           "sort": [{
  "_geo_distance": {
       "location": location,
          "order": "asc",
           "unit": "km", 
  "distance_type": "plane" 
          }}],
                }
    
res = es.search(index='passenger', doc_type='rolling', body=geo_query, ignore=[404, 400])



In [542]:
print res['hits']['hits'][0]["_source"] if res['hits']['hits'] else False


{u'status': u'arrived', u'city': u'NYC', u'altdest2id': u'9/11 Memorial & Museum', u'altdest2': [40.7114, -74.0125], u'altdest1': [40.7794, -73.9632], u'altdest1id': u'The Metropolitan Museum of Art', u'destination': [40.7588, -73.9851], u'driver': 25, u'ctime': u'2016-10-11T05:31:24.189421Z', u'destinationid': u'Times Square', u'location': [40.7589, -73.985], u'ptime': 0, u'path': [[40.7887, -73.984], [40.8078, -73.9686], [40.7833, -73.9723], [40.7988, -73.9742], [40.7887, -73.984], [40.7887, -73.984], [40.7588, -73.9851], [40.7588, -73.9851], [40.7588, -73.9851]], u'id': 77, u'name': u'passenger_77'}


In [554]:
def retrievePassenger(pID, es):
    p = es.get(index='passenger', doc_type='rolling', id=pID, \
                   ignore=[404, 400])
    return p['_source'] if p['found'] else False


In [713]:
p1 = retrievePassenger(5, es)
p2 = retrievePassenger(100, es)

In [723]:
p1Dest = [(p1['destination'], p1['destinationid']), (p1['altdest1'], p1['altdest1id']), \
              (p1['altdest2'], p1['altdest2id'])]
p2Dest = [(p2['destination'], p2['destinationid']), (p2['altdest1'], p2['altdest1id']), \
              (p2['altdest2'], p2['altdest2id'])]

dest1 = set([p1['destinationid'], p1['altdest1id'], p1['altdest2id']])  
dest2 = set([p2['destinationid'], p2['altdest1id'], p2['altdest2id']])

def distance(dest, location): 
    return vincenty(Point(dest), Point(location)).miles

far = float("inf")
dest = False

In [724]:

for i in xrange(len(p1Dest)):
    if (p1Dest[i][1] in list((dest1 & dest2))) and distance(p1Dest[i][0], location) < far:
        far = distance(p1Dest[i][0], location)
        dest = p1Dest[i]

In [725]:
dest

([41.8789, -87.6359], u'Skydeck Chicago')

In [688]:
def distance(dest, location): 
        return vincenty(Point(dest), Point(location)).miles


In [727]:
distance = distance(location, res[-1]) + int(p1['_source']['trip_distance'])

KeyError: -1

In [728]:
res

{u'_shards': {u'failed': 0, u'successful': 20, u'total': 20},
 u'hits': {u'hits': [{u'_id': u'77',
    u'_index': u'passenger',
    u'_score': None,
    u'_source': {u'altdest1': [40.7794, -73.9632],
     u'altdest1id': u'The Metropolitan Museum of Art',
     u'altdest2': [40.7114, -74.0125],
     u'altdest2id': u'9/11 Memorial & Museum',
     u'city': u'NYC',
     u'ctime': u'2016-10-11T05:31:24.189421Z',
     u'destination': [40.7588, -73.9851],
     u'destinationid': u'Times Square',
     u'driver': 25,
     u'id': 77,
     u'location': [40.7589, -73.985],
     u'name': u'passenger_77',
     u'path': [[40.7887, -73.984],
      [40.8078, -73.9686],
      [40.7833, -73.9723],
      [40.7988, -73.9742],
      [40.7887, -73.984],
      [40.7887, -73.984],
      [40.7588, -73.9851],
      [40.7588, -73.9851],
      [40.7588, -73.9851]],
     u'ptime': 0,
     u'status': u'arrived'},
    u'_type': u'rolling',
    u'sort': [3.4199593266745287e-06]}],
  u'max_score': None,
  u'total': 8},
 

In [733]:
res = es.get(index='passenger', doc_type='rolling', id=3, ignore=[400,404])


In [781]:
q = { 'size': 0,
    "aggs" : {
        "avg_wait" : { "filter" : {  "range" : { "ctime" : { "gt" : window,}}},
        "aggs" : {
            "avg_wait" : { "avg" : { "field" : "trip_distance" } }
            }}}}

res = es.search(index='passenger', doc_type='rolling', body=q, ignore=[404, 400])
if res['aggregations']['avg_wait']['avg_wait']['value']:
    res = round(float(res['aggregations']['avg_wait']['avg_wait']['value']), 2)
else:
    res = "N/A"
print res


KeyError: 'aggregations'

In [763]:
if res['aggregations']['avg_distance']['avg_distance']['value']:
        res = round(float(res['aggregations']['avg_distance']['avg_distance']['value']), 2)
else:
        res = 0

In [831]:
q = {'size': 5,  "fields" : ["id", "ptime", "trip_distance"],
        'query': { 'term': {'status': 'arrived'} },
        'filter': {'range': { 'ctime': { 'gt': window }} 
            },
         "sort": [{"ctime": {"order": "desc"}}]
        }
res = es.search(index='passenger', doc_type='rolling', body=q, ignore=[404, 400])
if res['hits']: print json.dumps(res['hits']['hits'])

[{"sort": [1476253264012], "_type": "rolling", "_index": "passenger", "_score": null, "fields": {"trip_distance": [4.932352479796495], "id": [5375], "ptime": [0]}, "_id": "5375"}, {"sort": [1476253262955], "_type": "rolling", "_index": "passenger", "_score": null, "fields": {"trip_distance": [2.7598842053010557], "id": [12127], "ptime": [0]}, "_id": "12127"}, {"sort": [1476253262625], "_type": "rolling", "_index": "passenger", "_score": null, "fields": {"trip_distance": [14.69976079428252], "id": [10424], "ptime": [0]}, "_id": "10424"}, {"sort": [1476253261561], "_type": "rolling", "_index": "passenger", "_score": null, "fields": {"trip_distance": [8.909810214656174], "id": [7117], "ptime": [0]}, "_id": "7117"}, {"sort": [1476253261042], "_type": "rolling", "_index": "passenger", "_score": null, "fields": {"trip_distance": [13.674180280802325], "id": [13512], "ptime": [0]}, "_id": "13512"}]


In [849]:
q = {'size': 5, 
    'query': { 'term': {'status': 'arrived'} },
    'filter': {'range': { 'ctime': { 'gt': window }} 
        },
     "sort": [{"ctime": {"order": "desc"}}]
    }
res = []
q = es.search(index='passenger', doc_type='rolling', body=q, ignore=[404, 400])
if q['hits']: res = json.dumps(q['hits']['hits'])
print res

[{"sort": [1476257366697], "_type": "rolling", "_source": {"status": "arrived", "trip_distance": 8.313455147356335, "destination": [41.7923, -87.5804], "altdest2id": "Adler Planetarium", "altdest2": [41.8663, -87.6068], "altdest1": [41.8663, -87.6171], "city": "CHI", "altdest1id": "The Field Museum", "driver": 2859, "ctime": "2016-10-12T07:29:26.697041Z", "destinationid": "Museum of Science and Industry", "location": [41.7924, -87.5803], "ptime": 0, "path": [[41.9059, -87.6338], [41.906, -87.6336], [41.7923, -87.5804]], "match": null, "id": 1067, "origin": [41.9059, -87.6338], "name": "passenger_1067"}, "_score": null, "_index": "passenger", "_id": "1067"}, {"sort": [1476257347544], "_type": "rolling", "_source": {"status": "arrived", "trip_distance": 48.268306750818944, "destination": [41.7923, -87.5804], "altdest2id": "Adler Planetarium", "altdest2": [41.8663, -87.6068], "altdest1": [41.8796, -87.6237], "city": "CHI", "altdest1id": "Art Institute of Chicago", "driver": 631, "ctime": 

In [858]:
r =  [i['_source'] for i in q['hits']['hits']]

In [857]:
r

[{u'altdest1': [41.8663, -87.6171],
  u'altdest1id': u'The Field Museum',
  u'altdest2': [41.8663, -87.6068],
  u'altdest2id': u'Adler Planetarium',
  u'city': u'CHI',
  u'ctime': u'2016-10-12T07:29:26.697041Z',
  u'destination': [41.7923, -87.5804],
  u'destinationid': u'Museum of Science and Industry',
  u'driver': 2859,
  u'id': 1067,
  u'location': [41.7924, -87.5803],
  u'match': None,
  u'name': u'passenger_1067',
  u'origin': [41.9059, -87.6338],
  u'path': [[41.9059, -87.6338], [41.906, -87.6336], [41.7923, -87.5804]],
  u'ptime': 0,
  u'status': u'arrived',
  u'trip_distance': 8.313455147356335},
 {u'altdest1': [41.8796, -87.6237],
  u'altdest1id': u'Art Institute of Chicago',
  u'altdest2': [41.8663, -87.6068],
  u'altdest2id': u'Adler Planetarium',
  u'city': u'CHI',
  u'ctime': u'2016-10-12T07:29:07.544930Z',
  u'destination': [41.7923, -87.5804],
  u'destinationid': u'Museum of Science and Industry',
  u'driver': 631,
  u'id': 11775,
  u'location': [41.7924, -87.5803],
  u

In [875]:
q = {'size': 20,
    'query': { 'terms': {'status': ['ontrip', 'arrived', 'pickup']} },
    'filter': {'range': { 'ctime': { 'gt': window }} 
        },
     "sort": [{"ctime": {"order": "desc"}}]
    }
res = []
q = es.search(index='passenger', doc_type='rolling', body=q, ignore=[404, 400])

In [876]:
[ i['trip_distance'] for i in [i['_source'] for i in q['hits']['hits']]]

[0.04184738843337838,
 0.03606014035509266,
 0.04942976198540751,
 0.029251639822103045,
 0.020738861111733755,
 0.2524479700340077,
 0.04218421230767893,
 0.017234265286246754,
 0.008615355481699923,
 0.8359894580153394,
 0.03165024528213993,
 0.03781868015341891,
 1.0436381645206783,
 0.11884283982884647,
 0.5325258725335308,
 0.045459327546630394,
 0.05295835851885678,
 0.11105083066425393,
 0.006901605531277425,
 0.041862865238358506]